In [1]:
import pandas as pd
from tqdm import tqdm

from joblib import Parallel, delayed
import multiprocessing as mp
from multiprocessing import cpu_count
from sklearn.model_selection import StratifiedGroupKFold, KFold
import json

import tensorflow as tf
cpu_count()

8

In [2]:
train_df = pd.read_csv('train.csv')
train_df = train_df.loc[(train_df["sequence_id"] == 0)].reset_index(drop = True)
with open('sign_to_prediction_index_map_merge.json') as json_file:
    LABEL_DICT = json.load(json_file)
    
print(LABEL_DICT)
#train_df = train_df.loc[(train_df["sign"].str.len() == 1)].reset_index(drop=True)

# Randomly select 5 rows for each unique "sign"
#train_df = train_df.groupby("sign").apply(lambda x: x.sample(n=min(25, len(x)))).reset_index(drop=True)

train_df

{'A': 0, 'B': 1, 'C': 2, 'D': 3, 'E': 4, 'F': 5, 'G': 6, 'H': 7, 'I': 8, 'J': 9, 'K': 10, 'L': 11, 'M': 12, 'N': 13, 'O': 14, 'P': 15, 'Q': 16, 'R': 17, 'S': 18, 'T': 19, 'U': 20, 'V': 21, 'W': 22, 'X': 23, 'Y': 24, 'Z': 25, 'TV': 26, 'after': 27, 'airplane': 28, 'all': 29, 'alligator': 30, 'animal': 31, 'another': 32, 'any': 33, 'apple': 34, 'arm': 35, 'aunt': 36, 'awake': 37, 'backyard': 38, 'bad': 39, 'balloon': 40, 'bath': 41, 'because': 42, 'bed': 43, 'bedroom': 44, 'bee': 45, 'before': 46, 'beside': 47, 'better': 48, 'bird': 49, 'black': 50, 'blow': 51, 'blue': 52, 'boat': 53, 'book': 54, 'boy': 55, 'brother': 56, 'brown': 57, 'bug': 58, 'bye': 59, 'callonphone': 60, 'can': 61, 'car': 62, 'carrot': 63, 'cat': 64, 'cereal': 65, 'chair': 66, 'cheek': 67, 'child': 68, 'chin': 69, 'chocolate': 70, 'clean': 71, 'close': 72, 'closet': 73, 'cloud': 74, 'clown': 75, 'cow': 76, 'cowboy': 77, 'cry': 78, 'cut': 79, 'cute': 80, 'dad': 81, 'dance': 82, 'dirty': 83, 'dog': 84, 'doll': 85, 'don

,path,participant_id,sequence_id,sign
0,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A
1,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A
2,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A
3,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A
4,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A
...,...,...,...,...
39241,train_landmark_files\1\Z_asl-sign-language-alp...,1,0.0,Z
39242,train_landmark_files\1\Z_asl-sign-language-alp...,1,0.0,Z
39243,train_landmark_files\1\Z_asl-sign-language-alp...,1,0.0,Z
39244,train_landmark_files\1\Z_asl-sign-language-alp...,1,0.0,Z


In [3]:
import numpy as np
import json

def load_relevant_data_subset(pq_path):
    data_columns = ['x', 'y', 'z']
    data = pd.read_parquet(pq_path, columns=data_columns)
    n_frames = int(len(data) / ROWS_PER_FRAME)
    data = data.values.reshape(n_frames, ROWS_PER_FRAME, len(data_columns))
    return data.astype(np.float32)

def encode_row(row):
    coordinates = load_relevant_data_subset(f'data/asl-signs/{row.path}')
    coordinates_encoded = coordinates.tobytes()
    participant_id = int(row.participant_id)
    sequence_id = int(row.sequence_id)
    sign = int(LABEL_DICT[row.sign])
    record_bytes = tf.train.Example(features=tf.train.Features(feature={
                'coordinates': tf.train.Feature(bytes_list=tf.train.BytesList(value=[coordinates_encoded])),
                'participant_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
                'sequence_id':tf.train.Feature(int64_list=tf.train.Int64List(value=[sequence_id])),
                'sign':tf.train.Feature(int64_list=tf.train.Int64List(value=[sign])),
                })).SerializeToString()
    return record_bytes

def process_chunk(chunk, tfrecord_name):
    options = tf.io.TFRecordOptions(compression_type='GZIP', compression_level=9)
    with tf.io.TFRecordWriter(tfrecord_name, options=options) as file_writer:
        for i, row in tqdm(chunk.iterrows()):
            record_bytes = encode_row(row)
            file_writer.write(record_bytes)
            del record_bytes
        file_writer.close()

ROWS_PER_FRAME = 543
N_FILES = len(train_df)
CHUNK_SIZE = 512
N_PART = 1
FOLD = 4
part = 0

class CFG:
    seed = 42
    n_splits = 5

row = train_df.iloc[0]
coordinates = load_relevant_data_subset(f'data/asl-signs/{row.path}')
coordinates_encoded = coordinates.tobytes()
participant_id = int(row.participant_id)
sequence_id = int(row.sequence_id) if pd.notna(row.sequence_id) else 0
sign = int(LABEL_DICT[row.sign])

record_bytes = tf.train.Example(features=tf.train.Features(feature={
            'coordinates': tf.train.Feature(bytes_list=tf.train.BytesList(value=[coordinates_encoded])),
            'participant_id': tf.train.Feature(int64_list=tf.train.Int64List(value=[participant_id])),
            'sequence_id':tf.train.Feature(int64_list=tf.train.Int64List(value=[sequence_id])),
            'sign':tf.train.Feature(int64_list=tf.train.Int64List(value=[sign])),
            }))

train_folds = train_df.copy()
train_folds['fold']=-1

num_bins = 5

# train_folds = train_folds.sample(frac=1, random_state=CFG.seed).reset_index(drop=True)
# gkfold = StratifiedGroupKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed) 
# print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
# for fold_idx, (train_idx, valid_idx) in enumerate(gkfold.split(train_folds, y=train_folds['sign'].values, groups=train_folds.participant_id)):
#     train_folds.loc[valid_idx,'fold'] = fold_idx
#     print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
kfold = KFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed) 
print(f'{CFG.n_splits}fold training', len(train_folds), 'samples')
for fold_idx, (train_idx, valid_idx) in enumerate(kfold.split(train_folds)):
    train_folds.loc[valid_idx,'fold'] = fold_idx
    print(f'fold{fold_idx}:', 'train', len(train_idx), 'valid', len(valid_idx))
    
assert not (train_folds['fold']==-1).sum()
assert len(np.unique(train_folds['fold']))==CFG.n_splits
train_folds.head()

5fold training 39246 samples
fold0: train 31396 valid 7850
fold1: train 31397 valid 7849
fold2: train 31397 valid 7849
fold3: train 31397 valid 7849
fold4: train 31397 valid 7849


,path,participant_id,sequence_id,sign,fold
0,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A,1
1,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A,2
2,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A,3
3,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A,2
4,train_landmark_files\1\A_asl-alphabet_asl_alph...,1,0.0,A,0


In [4]:
# Put every image in a seperate TFRecord file
# Make Pairs of Views as input to the model

def split_dataframe(df, chunk_size = 10000): 
    chunks = list()
    num_chunks = len(df) // chunk_size + 1
    for i in range(num_chunks):
        chunks.append(df[i*chunk_size:(i+1)*chunk_size])
    return chunks

#DATASET_NAME = f'islr-{CFG.n_splits}fold'
DATASET_NAME = f'fslr-{CFG.n_splits}fold'
#DATASET_NAME = f'aslr-{CFG.n_splits}fold'

for fold in range(CFG.n_splits):#[FOLD]:#range(CFG.n_splits):
    rows = train_folds[train_folds['fold']==fold]
    chunks = split_dataframe(rows, CHUNK_SIZE)
    part_size = len(chunks)//N_PART
    last = (part+1)*part_size if part != N_PART - 1 else len(chunks)+1
    chunks = chunks[part*part_size:last]
    
    N = [len(x) for x in chunks]
    _ = Parallel(n_jobs=cpu_count())(
        delayed(process_chunk)(x, f'data/{DATASET_NAME}/fold{fold}-{i}-{n}.tfrecords')
        for i,(x,n) in enumerate(zip(chunks,N))
    )